# 과제 : 에어비엔비 가격 예측 튜토리얼 최적 모델 결정
  - IT융합대학 컴퓨터공학과, 20134888 임형열
  - KNN : 최적 모델의 RMSE 값과 k값, 특징값 조합 구하기
  - Linear Regression : 최적모델의 RMSE 값과 특징값 조합 구하기
  - RMSE 비교하여 어느 모델이 더 좋은 성능인지 결정
  - 최종수정일 2020. 6. 9(화), 5:57PM

In [1]:
# k-nearest neighbor algorithm(k-최근접 이웃법 알고리즘) 사용
# 단순히 가까운 것을 선택하는 것이 아닌 주변의 몇 개(k개)를 보고 가장 많은 것을 골라내는 것

import pandas as pd
dc_listings = pd.read_csv('dc_airbnb.csv')

print(dc_listings.shape) # 약 3720개의 호스팅과 19개의 특징값이 출력됨 : (3720, 19)
dc_listings.head()

(3720, 19)


,host_response_rate,host_acceptance_rate,host_listings_count,accommodates,room_type,bedrooms,bathrooms,beds,price,cleaning_fee,security_deposit,minimum_nights,maximum_nights,number_of_reviews,latitude,longitude,city,zipcode,state
0,92%,91%,26,4,Entire home/apt,1.0,1.0,2,$160.00,$115.00,$100.00,1,1125,0.0,38.890046,-77.0028081,Washington,20003,DC
1,90%,100%,1,6,Entire home/apt,3.0,3.0,3,$350.00,$100.00,NaN,2,30,65.0,38.880413,-76.99048487,Washington,20003,DC
2,90%,100%,2,1,Private room,1.0,2.0,1,$50.00,NaN,NaN,2,1125,1.0,38.955291,-76.98600574,Hyattsville,20782,MD
3,100%,NaN,1,2,Private room,1.0,1.0,1,$95.00,NaN,NaN,1,1125,0.0,38.872134,-77.01963879,Washington,20024,DC
4,92%,67%,1,4,Entire home/apt,1.0,1.0,1,$50.00,$15.00,$450.00,7,1125,0.0,38.996382,-77.04154101,Silver Spring,20910,MD


In [2]:
# 1단계 : 비교하고 싶은 유사한 목록의 갯수(k)를 무엇으로 할지 결정
# 2단계 : 유사도 기준에 따라 우리 데이터와 모든 데이터간 유사도를 계산
# 유클리디안 거리, 절댓값

import numpy as np
our_acc_value = 3 # 숙박 가능한 인원을 최대 3명으로
 
first_living_space_value = dc_listings.loc[0,'accommodates'] # 열과 행. 첫 번째 호텔의 수용인원
first_distance = np.abs(int(first_living_space_value) - our_acc_value) # 나와 데이터 간 거리
# abs = 절댓값 사용

print(first_distance)

1


In [3]:
####
# str-int 불가능함 - 한 값이 private이라는 str 값으로 존재, 오류발생
# 해결방법 : 문자열 있는 행을 삭제 : 다른 데이터 또한 정확하게 기입되어있지 않았으므로
#dc_listings['distance'] = np.abs(dc_listings.accommodates - our_acc_value)
#dc_listings.distance.value_counts().sort_index()

####
# 문자열 숫자가 있는것으로 생각, 문자열을 숫자로 바꿔주는 기능을 넣음(to_numeric)
# private room을 숫자로 바꿀 수 없다는 오류발생
#dc_listings['distance'] = np.abs(dc_listings.accommodates.apply(pd.to_numeric) - our_acc_value)
#dc_listings.distance.value_counts().sort_index()

# private room 있는 행 삭제
dc_listings['distance'] = np.abs(dc_listings.accommodates.apply(pd.to_numeric, errors = 'coerce') - our_acc_value)
dc_listings.distance.value_counts().sort_index()
# out의 첫 번째 컬럼의 값이 작을수록 우리 데이터와 가깝다는 것을 의미함(ex) 0.0 = 우리 데이터와 같음)

0      461
1     2290
2      504
3      279
4       35
5       73
6       17
7       22
8        7
9       12
10       2
11       4
12       6
13       8
Name: distance, dtype: int64

In [4]:
# KNN 알고리즘

# sample() : 데이터 순서 랜덤하게 선택(셔플)
# frac = 1 : 모든 데이터에 대해(셔플할 데이터 갯수)
# random_state : 랜덤시드값(랜덤하게 섞는 방법?)
dc_listings = dc_listings.sample(frac=1,random_state=0)

#거리 기준으로 정렬
dc_listings = dc_listings.sort_values('distance')

dc_listings.price.head()
#dc_listings.price

2722     $85.00 
950      $99.00 
823     $139.00 
2162     $58.00 
710      $89.00 
Name: price, dtype: object

In [5]:
# 가격 열의 모든 데이터에 대하여 문자열을 실수형(float)으로 변환 (원래 데이터 타입은 object)
# 문자열 내부의 '\', '$', '|'는 ''로 대체함
dc_listings['price'] = dc_listings.price.str.replace("\$|,",'').astype(float)

# iloc[:5] : price열의 상위 5개에 대한 평균 값을 계산
mean_price = dc_listings.price.iloc[:5].mean()

mean_price

94.0

In [6]:
# 훈련셋/테스트셋으로 데이터를 분할, 성능 분석

dc_listings.drop('distance',axis=1) # axis : 열 갯수를 의미

# sklearn(사이킷런)을 쓰지 않고 임의로 훈련 셋과 테스트 셋을 나눔
train_df = dc_listings.copy().iloc[:2792] # 약 75%
test_df = dc_listings.copy().iloc[2792:] # 약 25%

In [7]:
# 두 개의 parameter를 갖는 함수 정의
# predict_price(test_df의 accommodate(수용인원), 들어간 수용인원 값이 위치하는 column) 
def predict_price(new_listing_value,feature_column):
    temp_df = train_df
    #temp_df['distance'] = np.abs(dc_listings[feature_column] - new_listing_value)
    #temp_df['distance'] = np.abs(temp_df[feature_column] - new_listing_value)
    temp_df['distance'] = np.abs(temp_df[feature_column].apply(pd.to_numeric, errors = 'coerce') - new_listing_value)
    
    # 거리순으로 정렬(가장 가까운 것이 최상위로)
    temp_df = temp_df.sort_values('distance')
    
    # 상위 5개에 대한 평균 값을 계산
    knn_5 = temp_df.price.iloc[:5]
    predicted_price = knn_5.mean()
    
    return(predicted_price)

In [8]:
# 함수 실행 : 모든 값에 대한 예측(각 숙박시설에 대한 예측값)
test_df['predicted_price'] = test_df.accommodates.apply(predict_price,feature_column='accommodates')
#test_df['predicted_price'] = test_df.bedrooms.apply(predict_price,feature_column='bedrooms')
test_df['predicted_price']

2214     72.0
1589     72.0
437      72.0
1198     72.0
3492    375.6
        ...  
611     375.6
763     375.6
1818    375.6
1596    375.6
1594    375.6
Name: predicted_price, Length: 928, dtype: float64

In [9]:
# RMSE 계산(root mean square error) : (예측값과 실제값의 차이)^2의 평균 - 루트 씌우기
# 이상치에 가까울수록 0에 근접
# 수치가 비정상적으로 클 경우 비정상적인 데이터모델(특징값)
# 혹은 비정상적인 알고리즘을 사용하지는 않았는지 확인해야 함

# accommodate에 대한 편차 : 218(상당히 큼) - 모델이 정확하지 않음, 사용한 특징값이 적절하지 않음

test_df['squared_error'] = (test_df['predicted_price'] - test_df['price'])**(2)

mse = test_df['squared_error'].mean()
rmse = mse ** (1/2)

rmse

221.51181675532257

In [10]:
# 각각의 특징값에 대한 RMSE를 계산하는 부분
# 수치(편차)가 작을수록 좋음
# 오류 발생(문자열을 숫자형으로 바꿀 수 없음) : csv 파일을 열어서 확인한 결과 3283열에 이상한 값 들어가있음, 삭제

for feature in ['accommodates','bedrooms','bathrooms','number_of_reviews']:

    #test_df['predicted_price'] = test_df.accommodates.apply(predict_price,feature_column=feature)
    test_df['predicted_price'] = test_df[feature].apply(predict_price,feature_column=feature)
    test_df['squared_error'] = (test_df['predicted_price'] - test_df['price'])**(2)
    
    mse = test_df['squared_error'].mean()
    rmse = mse ** (1/2)
    
    print("RMSE for the {} column: {}".format(feature,rmse)) # 각각의 특징값에 대한 RMSE값 출력

RMSE for the accommodates column: 221.51181675532257
RMSE for the bedrooms column: 208.99357076560838
RMSE for the bathrooms column: 227.55820849274423
RMSE for the number_of_reviews column: 241.35586281150165


# 정규화 된 파일 로드 후 분석

- 훈련 셋/테스트 셋 구분

In [11]:
normalized_listings = pd.read_csv('dc_airbnb.normalized.3.csv')

print(normalized_listings.shape) # 행과 열 갯수 출력 : 3666행, 8열
normalized_listings.head() # 파일의 위에서부터 5번째 부분까지 출력(기본값 5개)

(3666, 8)


,accommodates,bedrooms,bathrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews
0,0.400809,-0.249151,-0.439068,0.297181,160,-0.341333,-0.016584,-0.516903
1,1.398342,2.129464,2.969348,1.141072,350,-0.065163,-0.016615,1.705150
2,-1.095491,-0.249151,1.265140,-0.546711,50,-0.065163,-0.016584,-0.482718
3,-0.596724,-0.249151,-0.439068,-0.546711,95,-0.341333,-0.016584,-0.516903
4,0.400809,-0.249151,-0.439068,-0.546711,50,1.315689,-0.016584,-0.516903


In [12]:
# KNN 알고리즘

# sample() : 데이터 순서 랜덤하게 선택(셔플)
# frac = 1 : 모든 데이터에 대해(셔플할 데이터 갯수)
# random_state : 랜덤시드값(랜덤하게 섞는 방법)
normalized_listings = normalized_listings.sample(frac=1,random_state=0)

# 훈련 셋과 테스트 셋을 구분
norm_train_df = normalized_listings.copy().iloc[0:2750] # 약 75%
norm_test_df = normalized_listings.copy().iloc[2750:] # 약 25%

In [13]:
from scipy.spatial import distance # scipy의 distance 모듈 import, 우리 데이터와의 거리(가까움/멀음 여부) 판단

# iloc[0], [1] - 0번째와 1번째 선택
first_listing = normalized_listings.iloc[0][['accommodates', 'bathrooms']]
fifth_listing = normalized_listings.iloc[1][['accommodates', 'bathrooms']]

# distance의 유클리디안 거리(2개의 데이터에 대한)
first_fifth_distance = distance.euclidean(first_listing, fifth_listing)

first_fifth_distance

1.9950665769999998

In [14]:
# 두 개의 parameter를 갖는 함수 정의
# predict_price(test_df의 accommodate(수용인원), 들어간 수용인원 값이 위치하는 column) 
def predict_price_multivariate(new_listing_value,feature_columns):
    temp_df = norm_train_df
    temp_df['distance'] = distance.cdist(temp_df[feature_columns], [new_listing_value[feature_columns]])

    # 거리순으로 정렬(가장 가까운 것이 최상위로)
    temp_df = temp_df.sort_values('distance')
    
    # 상위 5개에 대한 평균 값을 계산
    knn_5 = temp_df.price.iloc[:5]
    predicted_price = knn_5.mean()
    
    return(predicted_price)

In [15]:
#MSE - RMSE 계산
cols = ['accommodates', 'bathrooms']

norm_test_df['predicted_price'] = norm_test_df[cols].apply(predict_price_multivariate, feature_columns=cols,axis=1)
norm_test_df['squared_error'] = (norm_test_df['predicted_price'] - norm_test_df['price'])**(2)

mse = norm_test_df['squared_error'].mean()
rmse = mse ** (1/2)

print(rmse)

97.26698255418684


In [16]:
# 사이킷런으로 KNN 알고리즘 사용

from sklearn.neighbors import KNeighborsRegressor
#knn = KNeighborsRegressor()

# algorithm 속성값
knn = KNeighborsRegressor(algorithm='brute')

In [17]:
# 훈련 특징값, 타겟(예측하고자 하는)
train_features = ['accommodates', 'bathrooms']
train_target = 'price'

knn.fit(norm_train_df[train_features], norm_train_df[train_target])

KNeighborsRegressor(algorithm='brute', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                    weights='uniform')

In [18]:
# 테스트 특징값
test_features = ['accommodates', 'bathrooms']

# 테스트 특징값으로 KNN 알고리즘을 활용, 가격 예측
two_features_predictions = knn.predict(norm_test_df[test_features])

In [19]:
# MSE import
from sklearn.metrics import mean_squared_error

# 여러가지 특징값에 대한 RMSE 계산 : 2가지 특징값
two_features_mse = mean_squared_error(norm_test_df['price'], two_features_predictions)
two_features_rmse = two_features_mse ** (1/2)

print(two_features_rmse)

90.92265680044032


In [20]:
#knn = KNeighborsRegressor(algorithm='brute')
cols = ['accommodates','bedrooms','bathrooms','beds']

knn.fit(norm_train_df[cols], norm_train_df['price'])

four_features_predictions = knn.predict(norm_test_df[cols])

# 여러가지 특징값에 대한 RMSE 계산 : 4가지 특징값
four_features_mse = mean_squared_error(norm_test_df['price'], four_features_predictions)
four_features_rmse = four_features_mse ** (1/2)

print(four_features_rmse)

92.92935500601551


# 최적 모델 결정을 위한 함수 선언

- 1개의 특징값만 사용하는 경우 : predict_price(parameter1, parameter2)
- 2개 이상의 특징값을 사용하는 경우 : predict_price_multivariate(parameter1, parameter2)

In [21]:
##### 함수 정의 : 1개의 특징값을 선택해서 사용할 경우 이용 #####
# k = {1, 3, 5} 중 하나의 값을 가지는 것으로 한정

k = 1 # k값
#k = [1, 3, 5]

def predict_price(new_listing_value,feature_column):
    temp_df = train_df
    temp_df['distance'] = np.abs(temp_df[feature_column].apply(pd.to_numeric, errors = 'coerce') - new_listing_value)
    
    # 거리순으로 정렬(가장 가까운 것이 최상위로)
    temp_df = temp_df.sort_values('distance')
    
    # 상위 k개에 대한 평균 값을 계산
    knn_value = temp_df.price.iloc[:k]
    predicted_price = knn_value.mean()
    
    return(predicted_price)

In [22]:
##### 함수 정의 : 2개 이상의 특징값을 선택해서 사용할 경우 이용 #####
# k = {1, 3, 5} 중 하나의 값을 가지는 것으로 한정

multik = 1 # k값
#multik = [1, 3, 5]

def predict_price_multivariate(new_listing_value,feature_columns):
    temp_df = norm_train_df
    temp_df['distance'] = distance.cdist(temp_df[feature_columns], [new_listing_value[feature_columns]])

    # 거리순으로 정렬(가장 가까운 것이 최상위로)
    temp_df = temp_df.sort_values('distance')
    
    # 상위 multik개에 대한 평균 값을 계산
    knn_value = temp_df.price.iloc[:multik]
    predicted_price = knn_value.mean()
    
    return(predicted_price)

In [23]:
### 1개의 특징값을 선택해서 사용할 경우 : 12가지(4가지 조합X3가지 k값)의 RSME 계산

for i in range(3) :
    for feature in ['accommodates','bedrooms','bathrooms','number_of_reviews']:
    
        #test_df['predicted_price'] = test_df.accommodates.apply(predict_price,feature_column=feature)
        test_df['predicted_price'] = test_df[feature].apply(predict_price,feature_column=feature)
        test_df['squared_error'] = (test_df['predicted_price'] - test_df['price'])**(2)
    
        mse = test_df['squared_error'].mean()
        rmse = mse ** (1/2)
    
        print("k값이 " + '[' + str(k) + ']' + "일때, [{}] 특징값으로 산출한 RMSE : {}".format(feature,rmse)) # 각각의 특징값에 대한 RMSE값 출력 
    
    k += 2
    
    if (k > 5) :
        k = 1


k값이 [1]일때, [accommodates] 특징값으로 산출한 RMSE : 211.34247782733203
k값이 [1]일때, [bedrooms] 특징값으로 산출한 RMSE : 197.34502064576785
k값이 [1]일때, [bathrooms] 특징값으로 산출한 RMSE : 244.51065103217266
k값이 [1]일때, [number_of_reviews] 특징값으로 산출한 RMSE : 237.49421159130242
k값이 [3]일때, [accommodates] 특징값으로 산출한 RMSE : 208.00742058652102
k값이 [3]일때, [bedrooms] 특징값으로 산출한 RMSE : 209.06243078095522
k값이 [3]일때, [bathrooms] 특징값으로 산출한 RMSE : 228.91157473903004
k값이 [3]일때, [number_of_reviews] 특징값으로 산출한 RMSE : 237.44013395387486
k값이 [5]일때, [accommodates] 특징값으로 산출한 RMSE : 221.51181675532257
k값이 [5]일때, [bedrooms] 특징값으로 산출한 RMSE : 208.99357076560838
k값이 [5]일때, [bathrooms] 특징값으로 산출한 RMSE : 227.55820849274423
k값이 [5]일때, [number_of_reviews] 특징값으로 산출한 RMSE : 241.35586281150165


In [24]:
### 2개의 특징값을 선택해서 사용할 경우 : 18가지(6가지 조합X3가지 k값)의 RSME 계산

cols1 = ['accommodates', 'bedrooms'] 
cols2 = ['accommodates', 'bathrooms']
cols3 = ['accommodates', 'number_of_reviews']
cols4 = ['bedrooms', 'bathrooms']
cols5 = ['bedrooms', 'number_of_reviews']        
cols6 = ['bathrooms', 'number_of_reviews']

multicols = [cols1, cols2, cols3, cols4, cols5, cols6]

for j in range(multik, 4) : 
    for i in multicols : 
        norm_test_df['predicted_price'] = norm_test_df[i].apply(predict_price_multivariate, feature_columns=i,axis=1)
        norm_test_df['squared_error'] = (norm_test_df['predicted_price'] - norm_test_df['price'])**(2)

        mse = norm_test_df['squared_error'].mean()
        rmse = mse ** (1/2)

        print("k값이 " + '[' + str(multik) + ']' + "일때, {} 특징값으로 산출한 RMSE : {}".format(i,rmse)) # 각각의 특징값에 대한 RMSE값 출력 
         
    multik += 2
    
    if (multik > 5) :
        multik = 1

k값이 [1]일때, ['accommodates', 'bedrooms'] 특징값으로 산출한 RMSE : 112.94650354816416
k값이 [1]일때, ['accommodates', 'bathrooms'] 특징값으로 산출한 RMSE : 111.77710001308965
k값이 [1]일때, ['accommodates', 'number_of_reviews'] 특징값으로 산출한 RMSE : 121.21624817226731
k값이 [1]일때, ['bedrooms', 'bathrooms'] 특징값으로 산출한 RMSE : 112.36360484749117
k값이 [1]일때, ['bedrooms', 'number_of_reviews'] 특징값으로 산출한 RMSE : 109.8702688743991
k값이 [1]일때, ['bathrooms', 'number_of_reviews'] 특징값으로 산출한 RMSE : 206.54291295232014
k값이 [3]일때, ['accommodates', 'bedrooms'] 특징값으로 산출한 RMSE : 95.76907724941908
k값이 [3]일때, ['accommodates', 'bathrooms'] 특징값으로 산출한 RMSE : 98.26592192584337
k값이 [3]일때, ['accommodates', 'number_of_reviews'] 특징값으로 산출한 RMSE : 108.79981416950494
k값이 [3]일때, ['bedrooms', 'bathrooms'] 특징값으로 산출한 RMSE : 88.02332241340815
k값이 [3]일때, ['bedrooms', 'number_of_reviews'] 특징값으로 산출한 RMSE : 120.260240616221
k값이 [3]일때, ['bathrooms', 'number_of_reviews'] 특징값으로 산출한 RMSE : 197.7652463742288
k값이 [5]일때, ['accommodates', 'bedrooms'] 특징값으로 산출한 RMSE : 91

In [25]:
### 3개의 특징값을 선택해서 사용할 경우 : 12가지(4가지 조합X3가지 k값)의 RSME 계산

cols1 = ['accommodates', 'bedrooms', 'bathrooms'] 
cols2 = ['accommodates', 'bedrooms', 'number_of_reviews']
cols3 = ['accommodates', 'bathrooms', 'number_of_reviews']
cols4 = ['bedrooms', 'bathrooms', 'number_of_reviews']

multicols = [cols1, cols2, cols3, cols4]

for j in range(multik, 4) : 
    for i in multicols : 
        norm_test_df['predicted_price'] = norm_test_df[i].apply(predict_price_multivariate, feature_columns=i,axis=1)
        norm_test_df['squared_error'] = (norm_test_df['predicted_price'] - norm_test_df['price'])**(2)

        mse = norm_test_df['squared_error'].mean()
        rmse = mse ** (1/2)

        print("k값이 " + '[' + str(multik) + ']' + "일때, {} 특징값으로 산출한 RMSE : {}".format(i,rmse)) # 각각의 특징값에 대한 RMSE값 출력 
         
    multik += 2
    
    if (multik > 5) :
        multik = 1

k값이 [1]일때, ['accommodates', 'bedrooms', 'bathrooms'] 특징값으로 산출한 RMSE : 99.3297571477039
k값이 [1]일때, ['accommodates', 'bedrooms', 'number_of_reviews'] 특징값으로 산출한 RMSE : 117.97851092169842
k값이 [1]일때, ['accommodates', 'bathrooms', 'number_of_reviews'] 특징값으로 산출한 RMSE : 111.63251177676474
k값이 [1]일때, ['bedrooms', 'bathrooms', 'number_of_reviews'] 특징값으로 산출한 RMSE : 96.64849312176277
k값이 [3]일때, ['accommodates', 'bedrooms', 'bathrooms'] 특징값으로 산출한 RMSE : 101.38111340513298
k값이 [3]일때, ['accommodates', 'bedrooms', 'number_of_reviews'] 특징값으로 산출한 RMSE : 92.78413289893304
k값이 [3]일때, ['accommodates', 'bathrooms', 'number_of_reviews'] 특징값으로 산출한 RMSE : 100.9839678668498
k값이 [3]일때, ['bedrooms', 'bathrooms', 'number_of_reviews'] 특징값으로 산출한 RMSE : 86.62125251735783
k값이 [5]일때, ['accommodates', 'bedrooms', 'bathrooms'] 특징값으로 산출한 RMSE : 91.27135246976142
k값이 [5]일때, ['accommodates', 'bedrooms', 'number_of_reviews'] 특징값으로 산출한 RMSE : 91.79599938005697
k값이 [5]일때, ['accommodates', 'bathrooms', 'number_of_reviews'] 특징값으

In [26]:
### 4개의 특징값을 선택해서 사용할 경우 : 3가지(1가지 조합X3가지 k값)의 RSME 계산

cols1 = ['accommodates', 'bedrooms', 'bathrooms', 'number_of_reviews']

for j in range(multik, 4) : 
    norm_test_df['predicted_price'] = norm_test_df[cols1].apply(predict_price_multivariate, feature_columns=cols1,axis=1)
    norm_test_df['squared_error'] = (norm_test_df['predicted_price'] - norm_test_df['price'])**(2)

    mse = norm_test_df['squared_error'].mean()
    rmse = mse ** (1/2)

    print("k값이 " + '[' + str(multik) + ']' + "일때, {} 특징값으로 산출한 RMSE : {}".format(cols1,rmse)) # 각각의 특징값에 대한 RMSE값 출력 
         
    multik += 2
    
    if (multik > 5) :
        multik = 1

k값이 [1]일때, ['accommodates', 'bedrooms', 'bathrooms', 'number_of_reviews'] 특징값으로 산출한 RMSE : 116.75719403136837
k값이 [3]일때, ['accommodates', 'bedrooms', 'bathrooms', 'number_of_reviews'] 특징값으로 산출한 RMSE : 97.261489332276
k값이 [5]일때, ['accommodates', 'bedrooms', 'bathrooms', 'number_of_reviews'] 특징값으로 산출한 RMSE : 93.64871856615534



# 최적 모델 결정(선형회귀)

In [27]:
from sklearn.linear_model import LinearRegression # 사이킷런에서 선형회귀 모듈 import
from sklearn.metrics import mean_squared_error # 사이킷런에서 MSE(mean_squared_error) 모듈 import 

In [35]:
# 훈련 셋과 테스트 셋을 구분 : 위의 코드에서 이미 실행하였으므로 주석처리함
#norm_train_df = normalized_listings.copy().iloc[0:2750] # 약 75%
#norm_test_df = normalized_listings.copy().iloc[2750:] # 약 25%

# 특징값 1개를 사용하여 RMSE 산출

train_target = 'price'

col = ['accommodates', 'bedrooms', 'bathrooms', 'number_of_reviews']

for i in col :
    linear = LinearRegression() # 선형회귀 사용
    # 모델 훈련 : coef_(기울기), intercept_(절편값) 전달
    # 독립변수 x에 대한 y의 값(상관성) 분석
    # 독립변수 x는 2차원 배열 형태여야 함 : 1가지 특징값만 사용할 경우 reshape(-1,1) 사용
    # np.array()형태로 데이터프레임의 특징값 형태 변환 후 reshape() 사용
    linear.fit(np.array(norm_train_df[i]).reshape(-1,1), norm_train_df[train_target]) # x는 독립변수, y는 종속변수
    
    # 어떠한 변수를 넣었을때 어느정도의 y값이 나올지? : 예측하는 부분
    # 해당 부분과 실제 존재하는(따로 테스트용 데이터로 뺀 25% 부분)의 차이(절댓값)가 rmse 
    predictions = linear.predict(np.array(norm_test_df[i]).reshape(-1,1))
    
    mse = mean_squared_error(norm_test_df[train_target], predictions)
    rmse = mse ** (1/2)

    print("{} 특징값으로 산출한 RMSE : {}".format(i,rmse)) # 각각의 특징값에 대한 RMSE값 출력     

accommodates 특징값으로 산출한 RMSE : 95.48246420254932
bedrooms 특징값으로 산출한 RMSE : 90.7456267465956
bathrooms 특징값으로 산출한 RMSE : 94.88266044114705
number_of_reviews 특징값으로 산출한 RMSE : 113.1974652557283


In [36]:
# 훈련 셋과 테스트 셋을 구분 : 위의 코드에서 이미 실행하였으므로 주석처리함
#norm_train_df = normalized_listings.copy().iloc[0:2750] # 약 75%
#norm_test_df = normalized_listings.copy().iloc[2750:] # 약 25%

# 특징값 2개를 동시에 사용하여 RMSE 산출

train_target = 'price'

cols1 = ['accommodates', 'bedrooms'] 
cols2 = ['accommodates', 'bathrooms']
cols3 = ['accommodates', 'number_of_reviews']
cols4 = ['bedrooms', 'bathrooms']
cols5 = ['bedrooms', 'number_of_reviews']        
cols6 = ['bathrooms', 'number_of_reviews']

multicols = [cols1, cols2, cols3, cols4, cols5, cols6]
 
for i in multicols : 
    linear = LinearRegression() # 선형회귀 사용
    # 모델 훈련 : coef_(기울기), intercept_(절편값) 전달
    # 독립변수 x에 대한 y의 값(상관성) 분석
    linear.fit(norm_train_df[i], norm_train_df[train_target]) # x는 독립변수, y는 종속변수
    
    # 어떠한 변수를 넣었을때 어느정도의 y값이 나올지? : 예측하는 부분
    # 해당 부분과 실제 존재하는(따로 테스트용 데이터로 뺀 25% 부분)의 차이(절댓값)가 rmse 
    predictions = linear.predict(norm_test_df[i])
    
    mse = mean_squared_error(norm_test_df[train_target], predictions)
    rmse = mse ** (1/2)

    print("{} 특징값으로 산출한 RMSE : {}".format(i,rmse)) # 각각의 특징값에 대한 RMSE값 출력 

['accommodates', 'bedrooms'] 특징값으로 산출한 RMSE : 89.42159100332559
['accommodates', 'bathrooms'] 특징값으로 산출한 RMSE : 88.72079063256996
['accommodates', 'number_of_reviews'] 특징값으로 산출한 RMSE : 95.02359364162102
['bedrooms', 'bathrooms'] 특징값으로 산출한 RMSE : 88.2320634292745
['bedrooms', 'number_of_reviews'] 특징값으로 산출한 RMSE : 90.67999577173926
['bathrooms', 'number_of_reviews'] 특징값으로 산출한 RMSE : 94.58388843541442


In [37]:
# 훈련 셋과 테스트 셋을 구분 : 위의 코드에서 이미 실행하였으므로 주석처리함
#norm_train_df = normalized_listings.copy().iloc[0:2750] # 약 75%
#norm_test_df = normalized_listings.copy().iloc[2750:] # 약 25%

# 특징값 3개를 동시에 사용하여 RMSE 산출
cols1 = ['accommodates', 'bedrooms', 'bathrooms'] 
cols2 = ['accommodates', 'bedrooms', 'number_of_reviews']
cols3 = ['accommodates', 'bathrooms', 'number_of_reviews']
cols4 = ['bedrooms', 'bathrooms', 'number_of_reviews']

train_target = 'price'

multicols = [cols1, cols2, cols3, cols4]
 
for i in multicols : 
    linear = LinearRegression()
    # 모델 훈련 : coef_(기울기), intercept_(절편값) 전달
    # 독립변수 x에 대한 y의 값(상관성) 분석
    linear.fit(norm_train_df[i], norm_train_df[train_target])
        
    # 어떠한 변수를 넣었을때 어느정도의 y값이 나올지? : 예측하는 부분
    # 해당 부분과 실제 존재하는(따로 테스트용 데이터로 뺀 25% 부분)의 차이(절댓값)가 rmse 
    predictions = linear.predict(norm_test_df[i])
    
    mse = mean_squared_error(norm_test_df[train_target], predictions)
    rmse = mse ** (1/2)

    print("{} 특징값으로 산출한 RMSE : {}".format(i,rmse)) # 각각의 특징값에 대한 RMSE값 출력 

['accommodates', 'bedrooms', 'bathrooms'] 특징값으로 산출한 RMSE : 86.87999334556163
['accommodates', 'bedrooms', 'number_of_reviews'] 특징값으로 산출한 RMSE : 89.27807239582494
['accommodates', 'bathrooms', 'number_of_reviews'] 특징값으로 산출한 RMSE : 88.35418915412448
['bedrooms', 'bathrooms', 'number_of_reviews'] 특징값으로 산출한 RMSE : 88.0977884397093


In [39]:
# 훈련 셋과 테스트 셋을 구분 : 위의 코드에서 이미 실행하였으므로 주석처리함
#norm_train_df = normalized_listings.copy().iloc[0:2750] # 약 75%
#norm_test_df = normalized_listings.copy().iloc[2750:] # 약 25%

# 특징값 4개를 모두 사용하여 RMSE 산출
cols = ['accommodates', 'bedrooms', 'bathrooms', 'number_of_reviews']
train_target = 'price'

linear = LinearRegression() # 선형회귀 사용
linear.fit(norm_train_df[cols], norm_train_df[train_target]) 
    
# 어떠한 변수를 넣었을때 어느정도의 y값이 나올지? : 예측하는 부분
# 해당 부분과 실제 존재하는(따로 테스트용 데이터로 뺀 25% 부분)의 차이(절댓값)가 rmse 
predictions = linear.predict(norm_test_df[cols])

mse = mean_squared_error(norm_test_df[train_target], predictions)
rmse = mse ** (1/2)

print("{} 특징값으로 산출한 RMSE : {}".format(cols,rmse))

['accommodates', 'bedrooms', 'bathrooms', 'number_of_reviews'] 특징값으로 산출한 RMSE : 86.67200445690683


 # 실험 코드
 
 - 데이터가 어떻게 들어가 있는지? 타입이랑 보기
 - 삭제해도 되고 안해도 됨.. 제출할땐 삭제

In [79]:
import sklearn.datasets

diabetes = sklearn.datasets.load_diabetes()

#diabetes
#type(diabetes['data'])
#dir(diabetes['data'])


diabetes['target']

array([151.,  75., 141., 206., 135.,  97., 138.,  63., 110., 310., 101.,
        69., 179., 185., 118., 171., 166., 144.,  97., 168.,  68.,  49.,
        68., 245., 184., 202., 137.,  85., 131., 283., 129.,  59., 341.,
        87.,  65., 102., 265., 276., 252.,  90., 100.,  55.,  61.,  92.,
       259.,  53., 190., 142.,  75., 142., 155., 225.,  59., 104., 182.,
       128.,  52.,  37., 170., 170.,  61., 144.,  52., 128.,  71., 163.,
       150.,  97., 160., 178.,  48., 270., 202., 111.,  85.,  42., 170.,
       200., 252., 113., 143.,  51.,  52., 210.,  65., 141.,  55., 134.,
        42., 111.,  98., 164.,  48.,  96.,  90., 162., 150., 279.,  92.,
        83., 128., 102., 302., 198.,  95.,  53., 134., 144., 232.,  81.,
       104.,  59., 246., 297., 258., 229., 275., 281., 179., 200., 200.,
       173., 180.,  84., 121., 161.,  99., 109., 115., 268., 274., 158.,
       107.,  83., 103., 272.,  85., 280., 336., 281., 118., 317., 235.,
        60., 174., 259., 178., 128.,  96., 126., 28

In [65]:
#norm = normalized_listings['accommodates']  
normarray = np.array(normalized_listings['accommodates'])

#type(normarray)
#type(normalized_listings['accommodates'])

numpy.ndarray